# AOC 2022

In [220]:
\cd notebooks

'io



# Advent of Code 2022 - 08

URL: https://adventofcode.com/2022/day/8

We'll study ngn's [solution](https://codeberg.org/ngn/k/src/branch/master/aoc/22/08.k):

This was the program that started my appetite for learning k. I couldn't believe that such an interesting problem could be solved in such a short codebase.

Tbh, at the start of writing this article, I still don't fully understand part2.
The nicest part of this solution is that the solution makes a lot of sense from the intuition point of view.

In [5]:
/ideas from https://codeberg.org/ovs/aoc2022/src/branch/master/08.k
/and https://github.com/chrispsn/aoc2022/blob/main/answers.ngnk
x:0:"i/08"
r:,/1|:\'1+:\x
R:,/{+y}'\{|y}\'2 2#
+//|/R@(-1>':|\)'r                 /part1
|//*/R@{x(+/1|':&\>/:)'1_(1_)\x}'r /part2

1849
201600


## Visibility of a row of trees

Let's start with a minimal example, and create a 1d forest, of 1 line of trees

In [2]:
:t:10?10

3 2 5 6 5 1 4 9 2 8


To find which trees are visible from the left, what we do is a max-scan, and compare each value with the max-so-far. If the current tree is higher, max-so-far is the value of the current tree, and we increase "visibles" by 1. 

Well. that, but in array mode :)

In [63]:
-1,'3 3#!10

(-1 0 1 2
 -1 3 4 5
 -1 6 7 8)


In [67]:
>':|\-1,'3 3#!10

(0 0 0 0
 0 1 1 1
 0 1 1 1)


In [4]:
1_>':|\-1,t 

1 0 1 1 0 0 0 1 0 0


A nicer way than prepending -1 and then cutting the first value, is to use the default value for `':` itself

In [3]:
-1>':|\t

1 0 1 1 0 0 0 1 0 0


In [10]:
:x:4 4#10?20

(0 5 3 6
 15 3 11 16
 2 17 0 5
 3 6 15 3)


## Rotate and repeat

The approach to solve this exercise is to solve the visibility problem for each one of the trees in the forest, and then, replicate the solution for each side of the forest.

The smart thing arraywise is to rotate the forest, and repeat the function, for each of the 4 orientations that result from rotating 90degrees.

Rotation of a table is quite easy in k. It's just a train of flip and reverse. The order of the operations dictates the way rotation.

In [5]:
:t:3 3#!9
+t /flip
|+t /flip+reverse = rotate
+|t /reverse+flip = rotate

(0 1 2
 3 4 5
 6 7 8)
(0 3 6
 1 4 7
 2 5 8)
(2 5 8
 1 4 7
 0 3 6)
(6 3 0
 7 4 1
 8 5 2)


In [7]:
+$(|+:)\t

(((,"0";,"1";,"2");(,"2";,"5";,"8");(,"8";,"7";,"6");(,"6";,"3";,"0"))
 ((,"3";,"4";,"5");(,"1";,"4";,"7");(,"5";,"4";,"3");(,"7";,"4";,"1"))
 ((,"6";,"7";,"8");(,"0";,"3";,"6");(,"2";,"1";,"0");(,"8";,"5";,"2")))


Now, here's a bit of a different solution from ngn's.

First we scan-rotate 3 times (keeping the original)

mv:(-1>':|\)'(+|:)\x

+//|/((!4) (|+:)/'mv)

In [12]:
:mv:(-1>':|\)'(+|:)\x

((1 1 1 1;1 0 1 1;0 1 0 0;0 0 1 0)
 (1 1 1 1;1 1 0 1;1 0 0 0;0 0 1 1)
 (1 1 1 1;1 0 1 0;1 0 0 1;0 0 0 0)
 (1 1 1 1;0 0 0 1;0 0 1 0;0 0 0 0))


We scan rotate `(+|:)` `x`. Remember that `monad scan` without an explicit number of iterations will stop when one iteration result is either the equal to the previous one, or equal to the initial argument. In this case, we know we'll end up with 4 iterations. After the 3rd iteration, the result would be the same as the original one.

A nice trick when we have 1 digit numbers in the matrices is to `flip` the result, to get a visual of how they look like in 2D.


In [15]:
$x

((,"0";,"5";,"3";,"6")
 ("15";,"3";"11";"16")
 (,"2";"17";,"0";,"5")
 (,"3";,"6";"15";,"3"))


In [22]:
$+(+|:)\x
`
$+mv

(((,"0";,"5";,"3";,"6");(,"3";,"2";"15";,"0");(,"3";"15";,"6";,"3");(,"6";"16";,"5";,"3"))
 (("15";,"3";"11";"16");(,"6";"17";,"3";,"5");(,"5";,"0";"17";,"2");(,"3";"11";,"0";"15"))
 ((,"2";"17";,"0";,"5");("15";,"0";"11";,"3");("16";"11";,"3";"15");(,"5";,"3";"17";,"6"))
 ((,"3";,"6";"15";,"3");(,"3";,"5";"16";,"6");(,"6";,"3";,"5";,"0");(,"0";"15";,"2";,"3")))
`
(((,"1";,"1";,"1";,"1");(,"1";,"1";,"1";,"1");(,"1";,"1";,"1";,"1");(,"1";,"1";,"1";,"1"))
 ((,"1";,"0";,"1";,"1");(,"1";,"1";,"0";,"1");(,"1";,"0";,"1";,"0");(,"0";,"0";,"0";,"1"))
 ((,"0";,"1";,"0";,"0");(,"1";,"0";,"0";,"0");(,"1";,"0";,"0";,"1");(,"0";,"0";,"1";,"0"))
 ((,"0";,"0";,"1";,"0");(,"0";,"0";,"1";,"1");(,"0";,"0";,"0";,"0");(,"0";,"0";,"0";,"0")))


This shows the different rotations and the trees that can be seen from the "north".

Next step is to add those numbers up. The intuitive operation is a max reduce over all the forest.

The problem here is that we can't just add the different rotated forests, because we would be doing the maxes of different trees.

For that, we have to "undo" the rotations, to make the trees align again.

In [33]:
(!4)(|+:)/'mv

((1 1 1 1;1 0 1 1;0 1 0 0;0 0 1 0)
 (1 1 0 1;1 0 0 1;1 1 0 0;1 1 1 0)
 (0 0 0 0;1 0 0 1;0 1 0 1;1 1 1 1)
 (0 0 0 1;0 0 0 1;0 1 0 1;0 0 1 1))


This is pretty neat: We're nesting an `each` with an `over monad`.

`l m'l`  where `m` is `(|+:)/`. 

For a simplified version, look at the following:

In [36]:
1 4 (100+)/'5 8

105 408


1 4 is the left `y` arguments. `5 8` is the right argument `x`. And the operation is a monad 